In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# filePath = "/mnt/raid/home/ayrisbud/us-famli-pl/src/new_annotated_data.csv"
myDf = pd.read_csv("/mnt/raid/home/ayrisbud/us-famli-pl/src/new_annotated_data.csv")
dfTrain = pd.read_csv("/mnt/raid/home/ayrisbud/us-famli-pl/src/annotatedTrain.csv")
dfTest = pd.read_csv("/mnt/raid/home/ayrisbud/us-famli-pl/src/annotatedTest.csv")
dfVal =pd.read_csv("/mnt/raid/home/ayrisbud/us-famli-pl/src/annotatedVal.csv")

In [3]:
myDf["Femur Visible"] = 0

In [4]:
dfMerged = pd.concat([dfTrain, dfVal], axis=0)
dfMerged = pd.concat([dfMerged, dfTest], axis=0)

In [5]:
dfMerged.head()
print(dfMerged.columns)
# myDf.head()
print(dfMerged["Amniotic fluid visible"].value_counts())
print(myDf.columns)

Index(['img_path', 'uuid', 'study_id', 'fetal_presentation_str', 'ga_boe',
       'ga_expert', 'famli_edd_source', 'famli_edd_source_detail', 'dvp_crf',
       'fetal_presentation', 'ga_absdiff', 'is_anchor', 'tag', 'pidscan',
       'us_plac', 'us_previa', 'placenta_str', 'previa_str', 'Manufacturer',
       'ManufacturerModelName', 'uuid_path_256', 'score', 'pred',
       'pred_cluster', 'File', 'No structures visible', 'Head Visible',
       'Abdomen Visible', 'Chest Visible', 'Femur Visible',
       'Other arm or leg bones visible', 'Umbilical cord visible',
       'Amniotic fluid visible', 'Placenta visible', 'Fetus or CRL visible',
       'Maternal bladder visible', 'Head Measurable', 'Abdomen Measurable',
       'Femur Measurable', 'Transverse plane for Head',
       'Longituidnal plane for Head', 'Oblique or other plane for Head',
       'Transverse plane for Abdomen', 'Longituidnal plane for Abdomen',
       'Oblique or other plane for Abdomen', 'Transverse plane for Femur',
 

In [6]:
myDfCols = myDf.columns
dfMergedCols = dfMerged.columns
print(len(myDfCols))
print(len(dfMergedCols))

32
48


In [7]:
myDf.rename(columns={"No_structures_visible": "No structures visible"}, inplace=True)
myDf.rename(columns={"Head_Visible": "Head Visible"}, inplace=True)
myDf.rename(columns={"Abdomen_Visible": "Abdomen Visible"}, inplace=True)
myDf.rename(columns={"Amniotic_fluid_visible": "Amniotic fluid visible"}, inplace=True)
myDf.rename(columns={"Placenta_visible": "Placenta visible"}, inplace=True)
myDf.rename(columns={"Fetus_or_CRL_visible": "Fetus or CRL visible"}, inplace=True)

In [8]:
myDf.columns
dfMerged.columns
myDfCols = myDf.columns
dfMergedCols = dfMerged.columns

In [9]:
temp3 = []
for element in myDfCols:
    if element not in dfMergedCols:
        temp3.append(element)
 
print(temp3)

['Unnamed: 0']


In [10]:
temp3 = []
for element in dfMergedCols:
    if element not in myDfCols:
        temp3.append(element)
 
print(temp3)

['pred_cluster', 'Chest Visible', 'Other arm or leg bones visible', 'Umbilical cord visible', 'Maternal bladder visible', 'Head Measurable', 'Abdomen Measurable', 'Femur Measurable', 'Transverse plane for Head', 'Longituidnal plane for Head', 'Oblique or other plane for Head', 'Transverse plane for Abdomen', 'Longituidnal plane for Abdomen', 'Oblique or other plane for Abdomen', 'Transverse plane for Femur', 'Longituidnal plane for Femur', 'Oblique or other plane for Femur']


In [11]:
dfMerged = dfMerged.drop(columns=temp3)


In [12]:
dfMergedFull = pd.concat([dfMerged, myDf], axis=0)

In [13]:
dfMergedFull.shape
dfMergedFull.columns

Index(['img_path', 'uuid', 'study_id', 'fetal_presentation_str', 'ga_boe',
       'ga_expert', 'famli_edd_source', 'famli_edd_source_detail', 'dvp_crf',
       'fetal_presentation', 'ga_absdiff', 'is_anchor', 'tag', 'pidscan',
       'us_plac', 'us_previa', 'placenta_str', 'previa_str', 'Manufacturer',
       'ManufacturerModelName', 'uuid_path_256', 'score', 'pred', 'File',
       'No structures visible', 'Head Visible', 'Abdomen Visible',
       'Femur Visible', 'Amniotic fluid visible', 'Placenta visible',
       'Fetus or CRL visible', 'Unnamed: 0'],
      dtype='object')

In [14]:
dfMergedFull = dfMergedFull.drop(dfMergedFull[dfMergedFull['Head Visible'] == "NotQCed"].index)

In [15]:
dfMergedFull.shape

(6582, 32)

In [16]:
dfMergedFull = dfMergedFull.drop(columns=["Unnamed: 0"], axis=1)

In [17]:
dfMergedFull = dfMergedFull.astype({'No structures visible':'int', 'Head Visible':'int', 'Abdomen Visible':'int', 'Amniotic fluid visible':'int', 'Placenta visible':'int', 'Fetus or CRL visible':'int'})

In [18]:
duplicate_values = dfMergedFull['uuid'].duplicated()
print(duplicate_values.value_counts())

uuid
False    6098
True      484
Name: count, dtype: int64


In [19]:
dfMergedFullNoDuplicates = dfMergedFull.drop_duplicates(subset=['uuid'])

In [20]:
dfMergedFullNoDuplicates.shape

(6098, 31)

In [21]:
from skmultilearn.model_selection import iterative_train_test_split

In [22]:
dfMergedFullNoDuplicates.head()

,img_path,uuid,study_id,fetal_presentation_str,ga_boe,ga_expert,famli_edd_source,famli_edd_source_detail,dvp_crf,fetal_presentation,...,score,pred,File,No structures visible,Head Visible,Abdomen Visible,Femur Visible,Amniotic fluid visible,Placenta visible,Fetus or CRL visible
0,extract_frames_blind_sweeps/dataset_C1_cines_m...,7a62a657-02de-47eb-ac49-4072f56da2f3,FAM-202-2430-1,Variable,67.167525,67.167525,ZAM-AnchorInTrim1,IG_CRL,4.0,4.0,...,1.053671e-07,142.69398,Groups/FAMLI/Shared/C1_ML_Analysis/extract_fra...,1,0,0,0,0,0,0
1,extract_frames_blind_sweeps/dataset_C1_cines_m...,d339a52a-007a-4a7e-b785-dfc473b0877a,FAM-202-2339-1,Transverse,120.840302,120.840302,ZAM-AnchorInTrim2,IG_HCFL,5.0,3.0,...,9.751744e-01,122.76802,Groups/FAMLI/Shared/C1_ML_Analysis/extract_fra...,0,1,0,0,1,1,1
2,extract_frames_blind_sweeps/dataset_C1_cines_m...,9b18e40a-86c4-45db-93b4-283beeaf64f4,FAM-202-1039-1,Cephalic,236.516779,236.516779,ZAM-AnchorInTrim3,IG_HCFL,4.0,1.0,...,7.406517e-04,255.12209,Groups/FAMLI/Shared/C1_ML_Analysis/extract_fra...,0,1,0,0,1,0,1
3,extract_frames_blind_sweeps/dataset_C1_cines_m...,c7fb026e-fdd7-4ab7-897f-727680444cbe,UNC-0554-5,Breech,197.000000,205.305842,UNC EPIC,LAST MENSTRUAL PERIOD,6.0,2.0,...,1.096368e-07,109.49991,Groups/FAMLI/Shared/C1_ML_Analysis/extract_fra...,1,0,0,0,0,0,0
4,extract_frames_blind_sweeps/dataset_C1_cines_m...,4f9d6380-25e0-473b-9c2b-5d031faf7174,UNC-0079-1,Cephalic,160.000000,0.000000,UNC EPIC,LAST MENSTRUAL PERIOD,8.0,1.0,...,9.647107e-01,159.43448,Groups/FAMLI/Shared/C1_ML_Analysis/extract_fra...,0,1,0,0,1,0,1


In [23]:
dfMergedFull1 = dfMergedFullNoDuplicates.reset_index(inplace=True)



In [24]:
dfMergedFullNoDuplicates.head()

,index,img_path,uuid,study_id,fetal_presentation_str,ga_boe,ga_expert,famli_edd_source,famli_edd_source_detail,dvp_crf,...,score,pred,File,No structures visible,Head Visible,Abdomen Visible,Femur Visible,Amniotic fluid visible,Placenta visible,Fetus or CRL visible
0,0,extract_frames_blind_sweeps/dataset_C1_cines_m...,7a62a657-02de-47eb-ac49-4072f56da2f3,FAM-202-2430-1,Variable,67.167525,67.167525,ZAM-AnchorInTrim1,IG_CRL,4.0,...,1.053671e-07,142.69398,Groups/FAMLI/Shared/C1_ML_Analysis/extract_fra...,1,0,0,0,0,0,0
1,1,extract_frames_blind_sweeps/dataset_C1_cines_m...,d339a52a-007a-4a7e-b785-dfc473b0877a,FAM-202-2339-1,Transverse,120.840302,120.840302,ZAM-AnchorInTrim2,IG_HCFL,5.0,...,9.751744e-01,122.76802,Groups/FAMLI/Shared/C1_ML_Analysis/extract_fra...,0,1,0,0,1,1,1
2,2,extract_frames_blind_sweeps/dataset_C1_cines_m...,9b18e40a-86c4-45db-93b4-283beeaf64f4,FAM-202-1039-1,Cephalic,236.516779,236.516779,ZAM-AnchorInTrim3,IG_HCFL,4.0,...,7.406517e-04,255.12209,Groups/FAMLI/Shared/C1_ML_Analysis/extract_fra...,0,1,0,0,1,0,1
3,3,extract_frames_blind_sweeps/dataset_C1_cines_m...,c7fb026e-fdd7-4ab7-897f-727680444cbe,UNC-0554-5,Breech,197.000000,205.305842,UNC EPIC,LAST MENSTRUAL PERIOD,6.0,...,1.096368e-07,109.49991,Groups/FAMLI/Shared/C1_ML_Analysis/extract_fra...,1,0,0,0,0,0,0
4,4,extract_frames_blind_sweeps/dataset_C1_cines_m...,4f9d6380-25e0-473b-9c2b-5d031faf7174,UNC-0079-1,Cephalic,160.000000,0.000000,UNC EPIC,LAST MENSTRUAL PERIOD,8.0,...,9.647107e-01,159.43448,Groups/FAMLI/Shared/C1_ML_Analysis/extract_fra...,0,1,0,0,1,0,1


In [25]:
dfMergedFullNoDuplicates = dfMergedFullNoDuplicates.drop(columns=["index"], axis=1)

In [26]:
dfMergedFullNoDuplicates.tail()

,img_path,uuid,study_id,fetal_presentation_str,ga_boe,ga_expert,famli_edd_source,famli_edd_source_detail,dvp_crf,fetal_presentation,...,score,pred,File,No structures visible,Head Visible,Abdomen Visible,Femur Visible,Amniotic fluid visible,Placenta visible,Fetus or CRL visible
6093,extract_frames_blind_sweeps/dataset_C1_cines_m...,c710f902-6399-46d9-ace4-7867629567aa,FAM-025-1123-1,Cephalic,137.891208,137.891208,ZAM-AnchorInTrim2,IG_HCFL,5.0,1.0,...,0.746454,134.21468,Groups\FAMLI\Shared\C1_ML_Analysis\extract_fra...,0,0,1,0,1,1,0
6094,extract_frames_blind_sweeps/dataset_C1_cines_m...,1d94d3f2-6f44-4b64-beae-59f4937da43b,VIL-0238-4,Breech,134.000000,137.764452,UNC EPIC,ULTRASOUND,5.0,2.0,...,0.999953,144.19089,Groups\FAMLI\Shared\C1_ML_Analysis\extract_fra...,0,1,0,0,1,0,0
6095,extract_frames_blind_sweeps/dataset_C1_cines_m...,283e1aea-9e1b-42d2-9cba-5122389d3878,FAM-025-1749-1,Cephalic,138.465454,138.465454,ZAM-AnchorInTrim2,IG_HCFL,9.0,1.0,...,0.999658,132.22163,Groups\FAMLI\Shared\C1_ML_Analysis\extract_fra...,0,1,0,0,1,1,0
6096,extract_frames_blind_sweeps/dataset_C1_cines_m...,616637e4-131e-4931-bdd5-e66652fbd174,FAM-202-1437-1,Cephalic,159.044588,159.044588,ZAM-AnchorInTrim2,IG_HCFL,6.0,1.0,...,0.650407,159.48991,Groups\FAMLI\Shared\C1_ML_Analysis\extract_fra...,0,0,1,0,1,1,0
6097,extract_frames_blind_sweeps/dataset_C1_cines_m...,4ab5f2a1-df4c-45de-9480-89c87c486b7d,VIL-0478-1,Transverse,115.000000,116.774546,UNC EPIC,ULTRASOUND,7.0,3.0,...,0.862165,112.05151,Groups\FAMLI\Shared\C1_ML_Analysis\extract_fra...,0,1,0,0,1,0,0


In [27]:
print(dfMergedFullNoDuplicates["Head Visible"].value_counts())
print(dfMergedFullNoDuplicates["Abdomen Visible"].value_counts())
print(dfMergedFullNoDuplicates["Placenta visible"].value_counts())
print(dfMergedFullNoDuplicates["Femur Visible"].value_counts())
print(dfMergedFullNoDuplicates["No structures visible"].value_counts())
print(dfMergedFullNoDuplicates["Amniotic fluid visible"].value_counts())

Head Visible
0    3664
1    2434
Name: count, dtype: int64
Abdomen Visible
0    4826
1    1272
Name: count, dtype: int64
Placenta visible
0    3858
1    2240
Name: count, dtype: int64
Femur Visible
0    5743
1     355
Name: count, dtype: int64
No structures visible
0    5405
1     693
Name: count, dtype: int64
Amniotic fluid visible
1    4889
0    1209
Name: count, dtype: int64


In [28]:
dfMergedFullNoDuplicates1 = dfMergedFullNoDuplicates.drop(columns=["Amniotic fluid visible", "Fetus or CRL visible"])

In [29]:
femurAbdomenDf = pd.read_csv("/mnt/raid/home/ayrisbud/us-famli-pl/src/abdomenFemur.csv")

In [30]:
# finalMergedDf = pd.merge(dfMergedFullNoDuplicates1, femurAbdomenDf, on="uuid", how='left')
finalMergedDf = pd.concat([dfMergedFullNoDuplicates1, femurAbdomenDf], axis=0)

In [31]:
# duplicate_values1 = finalMergedDf[finalMergedDf.duplicated(subset='uuid')]
# # print(duplicate_values1.value_counts())
# print(duplicate_values1)

finalMergedDfNoDuplicates = finalMergedDf.drop_duplicates(subset=['uuid'])
finalMergedDfNoDuplicates = finalMergedDfNoDuplicates.drop(columns=['frame_num'])


In [33]:
print(finalMergedDfNoDuplicates.columns)
print(finalMergedDfNoDuplicates.shape)

Index(['img_path', 'uuid', 'study_id', 'fetal_presentation_str', 'ga_boe',
       'ga_expert', 'famli_edd_source', 'famli_edd_source_detail', 'dvp_crf',
       'fetal_presentation', 'ga_absdiff', 'is_anchor', 'tag', 'pidscan',
       'us_plac', 'us_previa', 'placenta_str', 'previa_str', 'Manufacturer',
       'ManufacturerModelName', 'uuid_path_256', 'score', 'pred', 'File',
       'No structures visible', 'Head Visible', 'Abdomen Visible',
       'Femur Visible', 'Placenta visible'],
      dtype='object')
(9024, 29)


In [34]:
dfMergedFullX = finalMergedDfNoDuplicates.iloc[:, 0:-5].values
dfMergedFullY = finalMergedDfNoDuplicates.iloc[:, -5:].values

In [35]:
print(type(dfMergedFullY))
print(dfMergedFullY.shape)

<class 'numpy.ndarray'>
(9024, 5)


In [36]:
finalMergedDfNoDuplicates.tail()


,img_path,uuid,study_id,fetal_presentation_str,ga_boe,ga_expert,famli_edd_source,famli_edd_source_detail,dvp_crf,fetal_presentation,...,ManufacturerModelName,uuid_path_256,score,pred,File,No structures visible,Head Visible,Abdomen Visible,Femur Visible,Placenta visible
2995,extract_frames_blind_sweeps/dataset_C1_cines_m...,157db2d9-1da7-4a71-8f9d-c017e3674135,FAM-202-1861-1,Cephalic,173.529306,173.529306,ZAM-AnchorInTrim2,IG_HCFL,5.0,1.0,...,Turbo,dataset_C1_cines_masked_resampled_256_spc075_u...,0.081351,172.99520,NaN,0,0,0,1,0
2996,extract_frames_blind_sweeps/dataset_C1_cines_m...,dfe4d887-21cc-44ec-b329-7aa59aa11de1,FAM-025-0558-3,Cephalic,255.115704,255.317601,ZAM-AnchorInTrim1,IG_CRL,5.0,1.0,...,Turbo,dataset_C1_cines_masked_resampled_256_spc075_u...,0.000966,264.65582,NaN,0,0,0,1,0
2997,extract_frames_blind_sweeps/dataset_C1_cines_m...,f96a2a59-1b65-4aa0-8236-f6abaeddaf29,FAM-202-0136-2,Cephalic,263.023468,NaN,ZAM-AnchorInTrim3,IG_HCFL,3.0,1.0,...,Voluson S,dataset_C1_cines_masked_resampled_256_spc075_u...,0.021443,247.24250,NaN,0,0,0,1,0
2998,extract_frames_blind_sweeps/dataset_C1_cines_m...,565c4449-3a59-4060-9d72-b447e53234c8,FAM-202-2110-1,Cephalic,143.986745,143.986745,ZAM-AnchorInTrim2,IG_HCFL,4.0,1.0,...,LOGIQe,dataset_C1_cines_masked_resampled_256_spc075_u...,0.088993,150.75090,NaN,0,0,0,1,0
2999,extract_frames_blind_sweeps/dataset_C1_cines_m...,7356e2de-46b4-4491-960b-bf6928617dea,FAM-025-0899-1,Cephalic,128.170238,128.170238,ZAM-AnchorInTrim2,IG_HCFL,4.0,1.0,...,Turbo,dataset_C1_cines_masked_resampled_256_spc075_u...,0.100813,129.62344,NaN,0,0,0,1,0


In [37]:
finalMergedDfNoDuplicates.dtypes



img_path                    object
uuid                        object
study_id                    object
fetal_presentation_str      object
ga_boe                     float64
ga_expert                  float64
famli_edd_source            object
famli_edd_source_detail     object
dvp_crf                    float64
fetal_presentation         float64
ga_absdiff                 float64
is_anchor                    int64
tag                         object
pidscan                     object
us_plac                    float64
us_previa                  float64
placenta_str                object
previa_str                  object
Manufacturer                object
ManufacturerModelName       object
uuid_path_256               object
score                      float64
pred                       float64
File                        object
No structures visible        int64
Head Visible                 int64
Abdomen Visible              int64
Femur Visible                int64
Placenta visible    

In [38]:
finalMergedDfNoDuplicates.dtypes

img_path                    object
uuid                        object
study_id                    object
fetal_presentation_str      object
ga_boe                     float64
ga_expert                  float64
famli_edd_source            object
famli_edd_source_detail     object
dvp_crf                    float64
fetal_presentation         float64
ga_absdiff                 float64
is_anchor                    int64
tag                         object
pidscan                     object
us_plac                    float64
us_previa                  float64
placenta_str                object
previa_str                  object
Manufacturer                object
ManufacturerModelName       object
uuid_path_256               object
score                      float64
pred                       float64
File                        object
No structures visible        int64
Head Visible                 int64
Abdomen Visible              int64
Femur Visible                int64
Placenta visible    

In [39]:
myTrainDfX, myTrainDfY, myTestDfX, myTestDfY = iterative_train_test_split(X=dfMergedFullX, y=dfMergedFullY, test_size=0.2)

In [40]:
myTrainDfX, myTrainDfY, myValDfX, myValDfY = iterative_train_test_split(X=myTrainDfX, y=myTrainDfY, test_size=0.2)

In [41]:
print(type(myTrainDfY))

<class 'numpy.ndarray'>


In [42]:
myTrainDf = pd.concat([pd.DataFrame(myTrainDfX, columns=finalMergedDfNoDuplicates.columns[:-5]), pd.DataFrame(myTrainDfY, columns=finalMergedDfNoDuplicates.columns[-5:])], axis=1)
myTestDf = pd.concat([pd.DataFrame(myTestDfX, columns=finalMergedDfNoDuplicates.columns[:-5]), pd.DataFrame(myTestDfY, columns=finalMergedDfNoDuplicates.columns[-5:])], axis=1)
myValDf = pd.concat([pd.DataFrame(myValDfX, columns=finalMergedDfNoDuplicates.columns[:-5]), pd.DataFrame(myValDfY, columns=finalMergedDfNoDuplicates.columns[-5:])], axis=1)

# myTestDf = pd.concat([myTestDfX, myTestDfY], axis=1)
# myValDf = pd.concat([myValDfX, myValDfY], axis=1)

In [43]:
print(myTrainDf.shape)
print(myTestDf.shape)
print(myValDf.shape)

(5775, 29)
(1805, 29)
(1444, 29)


In [45]:
print(myTrainDf["Head Visible"].value_counts())
print(myTrainDf["Abdomen Visible"].value_counts())
print(myTrainDf["Placenta visible"].value_counts())
print(myTrainDf["Femur Visible"].value_counts())
print(myTrainDf["No structures visible"].value_counts())

print(myTestDf["Head Visible"].value_counts())
print(myTestDf["Abdomen Visible"].value_counts())
print(myTestDf["Placenta visible"].value_counts())
print(myTestDf["Femur Visible"].value_counts())
print(myTestDf["No structures visible"].value_counts())

print(myValDf["Head Visible"].value_counts())
print(myValDf["Abdomen Visible"].value_counts())
print(myValDf["Placenta visible"].value_counts())
print(myValDf["Femur Visible"].value_counts())
print(myValDf["No structures visible"].value_counts())


Head Visible
0    4217
1    1558
Name: count, dtype: int64
Abdomen Visible
0    4342
1    1433
Name: count, dtype: int64
Placenta visible
0    4341
1    1434
Name: count, dtype: int64
Femur Visible
0    4294
1    1481
Name: count, dtype: int64
No structures visible
0    5332
1     443
Name: count, dtype: int64
Head Visible
0    1318
1     487
Name: count, dtype: int64
Abdomen Visible
0    1357
1     448
Name: count, dtype: int64
Placenta visible
0    1357
1     448
Name: count, dtype: int64
Femur Visible
0    1342
1     463
Name: count, dtype: int64
No structures visible
0    1666
1     139
Name: count, dtype: int64
Head Visible
0    1055
1     389
Name: count, dtype: int64
Abdomen Visible
0    1086
1     358
Name: count, dtype: int64
Placenta visible
0    1086
1     358
Name: count, dtype: int64
Femur Visible
0    1074
1     370
Name: count, dtype: int64
No structures visible
0    1333
1     111
Name: count, dtype: int64


In [44]:
myTrainDf.to_csv("/mnt/raid/home/ayrisbud/us-famli-pl/src/newAnnotatedTrainFinal.csv")
myTestDf.to_csv("/mnt/raid/home/ayrisbud/us-famli-pl/src/newAnnotatedTestFinal.csv")
myValDf.to_csv("/mnt/raid/home/ayrisbud/us-famli-pl/src/newAnnotatedValFinal.csv")

In [ ]:
myTrainDf.columns

In [ ]:
# myDf1 = myDf.replace(np.nan, "NO")
# myDf1.head()
myDf.head()

In [ ]:
myDf1 = myDf1.replace("NO", 0)
myDf1 = myDf1.replace("YES", 1)
myDf1.head()


In [ ]:
myDf1.columns
myLabels = ['Head Visible',
       'Abdomen Visible', 'Chest Visible', 'Femur Visible',
       'Other arm or leg bones visible', 'Umbilical cord visible',
       'Amniotic fluid visible', 'Placenta visible', 'Fetus or CRL visible',
       'Maternal bladder visible', 'Head Measurable', 'Abdomen Measurable',
       'Femur Measurable']

In [ ]:
train_df, test_df = train_test_split(myDf1, test_size=0.2)

In [ ]:
train_df, val_df = train_test_split(train_df, test_size=0.2)

In [ ]:
print(train_df.head())

In [ ]:
train_df.to_csv("./annotatedTrain.csv", index=False)
test_df.to_csv("./annotatedTest.csv", index=False)
val_df.to_csv("./annotatedVal.csv", index=False)

In [ ]:
myTrain_df = pd.read_csv("./annotatedTrain.csv")
myTrain_df.head()